<a href="https://colab.research.google.com/github/NiesHW/SECB3203_P4B/blob/main/Group_Project/Group_11/Final%20Report/DTI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Collect Drug and Target (Protein) Interaction**

In [ ]:
import os
import pandas as pd
import numba
import urllib


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)  # This will prompt for authentication

# Accessing a specific file
raw_folder = '/content/drive/My Drive/DTI Data/'
raw_files = {'drug_drug': 'mat_drug_drug.txt', 'drug_disease': 'mat_drug_disease.txt',
             'drug_sideeffect': 'mat_drug_se.txt', 'protein_disease': 'mat_protein_disease.txt',
             'protein_protein': 'mat_protein_protein.txt'}

raw_df = dict()
for key, val in raw_files.items():
    raw_df[key] = pd.read_csv(urllib.parse.urljoin(raw_folder, val), header=None, sep='\s+')
    print(f'{key} table shape: {raw_df[key].shape}')

Mounted at /content/drive
drug_drug table shape: (708, 708)
drug_disease table shape: (708, 5603)
drug_sideeffect table shape: (708, 4192)
protein_disease table shape: (1512, 5603)
protein_protein table shape: (1512, 1512)


**2. Generate a similarity matrix using Cosine Similarity**

In [ ]:
# add drug code, disease name
drug_name = pd.read_csv(urllib.parse.urljoin(raw_folder,'drug.txt'), header=None)
disease_name = pd.read_csv(urllib.parse.urljoin(raw_folder,'disease.txt'), header=None, sep='\t')

raw_df['drug_disease'].index = drug_name[0]; raw_df['drug_disease'].index.name =None
raw_df['drug_disease'].columns = disease_name[0]
raw_df['drug_disease'].iloc[:10,:10]

,depressive disorder,drug-induced liver injury,mercury poisoning,necrosis,neoplasms,"anemia, hemolytic",attention deficit and disruptive behavior disorders,autistic disorder,cognition disorders,cystitis
DB00050,1,1,1,1,1,0,0,1,1,0
DB00152,1,1,0,0,1,0,0,1,1,0
DB00162,1,1,0,1,1,1,0,1,1,1
DB00175,1,1,0,1,1,1,0,1,1,0
DB00176,1,1,0,1,0,0,1,1,0,0
DB00177,1,1,1,1,1,0,0,1,1,0
DB00178,1,1,0,1,1,0,0,1,1,0
DB00180,0,1,0,0,0,0,0,0,0,0
DB00181,1,1,0,1,0,0,0,1,1,0
DB00182,1,1,0,1,1,0,0,1,1,0


In [ ]:
import numpy as np
from sklearn.metrics import pairwise_distances
# Merge each drug and target similarity files
similarity_df = dict()
for key, val in raw_df.items():
  _df = val.astype(bool).to_numpy()
  _df = pd.DataFrame(pairwise_distances(_df, metric='cosine'))
  similarity_df[key] = _df + np.eye(len(_df))

similarity_df['drug_disease'].apply(lambda x: round(x,2))


,0,1,2,3,4,5,6,7,8,9,...,698,699,700,701,702,703,704,705,706,707
0,1.00,0.64,0.51,0.44,0.68,0.39,0.46,0.71,0.61,0.49,...,0.92,0.96,0.91,0.92,0.57,0.95,0.92,0.97,0.50,0.48
1,0.64,1.00,0.65,0.57,0.72,0.59,0.50,0.80,0.67,0.67,...,0.94,0.98,0.91,0.95,0.75,1.00,0.93,1.00,0.62,0.63
2,0.51,0.65,1.00,0.36,0.65,0.36,0.49,0.78,0.58,0.41,...,0.95,0.92,0.90,0.93,0.67,0.97,0.86,0.98,0.50,0.44
3,0.44,0.57,0.36,1.00,0.60,0.34,0.42,0.76,0.52,0.39,...,0.94,0.93,0.89,0.91,0.67,1.00,0.87,0.98,0.41,0.25
4,0.68,0.72,0.65,0.60,1.00,0.67,0.69,0.80,0.63,0.55,...,0.95,0.95,0.92,0.86,0.70,1.00,0.85,1.00,0.66,0.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,0.95,1.00,0.97,1.00,1.00,0.96,0.95,1.00,0.94,0.96,...,1.00,1.00,1.00,1.00,1.00,1.00,0.76,1.00,1.00,1.00
704,0.92,0.93,0.86,0.87,0.85,0.91,0.91,1.00,0.81,0.85,...,1.00,1.00,1.00,1.00,0.96,0.76,1.00,1.00,0.87,0.91
705,0.97,1.00,0.98,0.98,1.00,0.98,0.97,1.00,0.97,0.98,...,1.00,1.00,1.00,1.00,0.95,1.00,1.00,1.00,0.96,0.97
706,0.50,0.62,0.50,0.41,0.66,0.40,0.51,0.76,0.41,0.42,...,0.91,0.94,0.87,0.90,0.66,1.00,0.87,0.96,1.00,0.32


**3. Reduce similarity matrix dimensions using NMF**

In [ ]:
similarity_df['drug'] = pd.read_csv(urllib.parse.urljoin(raw_folder,'Similarity_Matrix_Drugs.txt'), header=None, sep='\s+')
similarity_df['protein'] = pd.read_csv(urllib.parse.urljoin(raw_folder,'Similarity_Matrix_Proteins.txt'), header=None, sep=' ')

In [ ]:
def calc_dca(df_mat, maxiter=20, restart_prob=0.5):

    mat = df_mat.div(df_mat.sum(axis=1),axis=0).to_numpy()
    restart = np.eye(len(mat))
    mat_ret = np.eye(len(mat))

    for i in range(maxiter):
        mat_ret_new = (1-restart_prob) * mat * mat_ret + restart_prob * restart
        delta = np.linalg.norm(mat_ret-mat_ret_new, ord='fro')
        mat_ret = mat_ret_new
        if delta < 1e-6:
            break

    return mat_ret

In [ ]:
drug_mat = ['drug_drug','drug_disease','drug_sideeffect','drug']
protein_mat = ['protein_disease','protein_protein','protein']

drug_dca = np.concatenate([calc_dca(similarity_df[key]) for key in drug_mat], axis=1)
protein_dca = np.concatenate([calc_dca(similarity_df[key]) for key in protein_mat], axis=1)

# Concatenate matrix of Drug similarity
pd.DataFrame(drug_dca).apply(lambda x: round(x, 2))

,0,1,2,3,4,5,6,7,8,9,...,2822,2823,2824,2825,2826,2827,2828,2829,2830,2831
0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0


In [ ]:
from sklearn.decomposition import NMF

# number of features
num_feature = 250

def calc_nmf(dca_mtx, num_feature=250):

  alpha = 1 / len(dca_mtx)
  dca_mtx = np.log(dca_mtx + alpha) - np.log(alpha)
  dca_mtx = np.matmul(dca_mtx, dca_mtx.transpose())

  nmf = NMF(n_components=num_feature)
  nmf.fit(dca_mtx)
  result = nmf.transform(dca_mtx)

  return result

In [ ]:
drug_nmf = calc_nmf(drug_dca, num_feature)
protein_nmf = calc_nmf(protein_dca, num_feature)

pd.DataFrame(drug_nmf).apply(lambda x: round(x,2))

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.02,0.00,0.03
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00
704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.03,0.0,0.0,0.0,0.00,0.00,0.00
705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00
706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00


**4. Matrix completion to search new drug-target interaction**

In [ ]:
import time
from numba import jit

@jit(nopython=True)
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    '''
    ORIGINAL SOURCE: https://towardsdatascience.com/recommendation-system-matrix-factorization-d61978660b4b
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''

    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = np.dot(P,Q)

        e = 0

        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))

        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [ ]:

R = pd.read_csv(urllib.parse.urljoin(raw_folder,'mat_drug_protein.txt'), header=None, sep='\s+').to_numpy()

# N: number of drugs
N = R.shape[0]
# M: number of proteins
M = R.shape[1]
# Number of feature
K = num_feature

start = time.time()
n_drug, n_protein = matrix_factorization(R, drug_nmf, protein_nmf, K)
end = time.time()
print(f'Elapse: {end-start}')

Elapse: 278.1758372783661


In [ ]:
n_drug_protein = np.dot(n_drug, n_protein.T)

**5. Evaluate predicted drug-target interaction**

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
rhat = n_drug_protein
rhat = pd.DataFrame(rhat).stack().reset_index()
rhat.set_index(rhat.apply(lambda x: f'{int(x.level_0)}_{int(x.level_1)}', axis=1), inplace=True)
rhat = rhat[0]
rhat.sort_index(inplace=True)

r = R
r = pd.DataFrame(r).stack().reset_index()
r.set_index(r.apply(lambda x: f'{int(x.level_0)}_{int(x.level_1)}', axis=1), inplace=True)
r = r[0]
r = r[rhat.index]
r.sort_index(inplace=True)

roc_auc_score(r, rhat)

0.9976720738570403